In [1]:
from model.mnist_image_plus_random_number import Net
import torch 
!pip install torchsummary
from torchsummary import summary
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
model = Net().to(device)
summary(model, input_size=(1, 28, 28))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1            [-1, 8, 28, 28]              72
              ReLU-2            [-1, 8, 28, 28]               0
       BatchNorm2d-3            [-1, 8, 28, 28]              16
            Conv2d-4           [-1, 16, 28, 28]           1,152
              ReLU-5           [-1, 16, 28, 28]               0
       BatchNorm2d-6           [-1, 16, 28, 28]              32
            Conv2d-7            [-1, 8, 28, 28]             136
         MaxPool2d-8            [-1, 8, 14, 14]               0
           Dropout-9            [-1, 8, 14, 14]               0
           Conv2d-10            [-1, 8, 14, 14]             576
             ReLU-11            [-1, 8, 14, 14]               0
      BatchNorm2d-12            [-1, 8, 14, 14]              16
           Conv2d-13           [-1, 16, 14, 14]           1,152
             ReLU-14           [-1, 16,

In [2]:
!nvidia-smi

Mon May 17 21:19:02 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 451.48       Driver Version: 451.48       CUDA Version: 11.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name            TCC/WDDM | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  GeForce GTX 1650   WDDM  | 00000000:01:00.0  On |                  N/A |
| N/A   50C    P8     3W /  N/A |    626MiB /  4096MiB |      3%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                                  |
|  GPU  

In [3]:
from torchvision import transforms
from dataloader.mnist_random_digit_dataloader import MnistImageWithRandomNumberDataset as MIRND
torch.manual_seed(1)
batch_size = 512

kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}

train_loader = torch.utils.data.DataLoader(
    MIRND(root_directory = './data', train=True,
                    transform=[
                        transforms.RandomRotation((-7.0, 7.0), fill=(1,)),
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ]),
    batch_size=batch_size, shuffle=True, **kwargs)
    
test_loader = torch.utils.data.DataLoader(
    MIRND(root_directory = './data', train=False, transform=[
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ]),
    batch_size=batch_size, shuffle=True, **kwargs)

In [4]:
from tqdm import tqdm
import torch.nn.functional as F

train_losses = []
test_losses = []
train_acc = []
test_acc = []

def train(model, device, train_loader, optimizer, epoch):
  model.train()
  pbar = tqdm(train_loader, position = 0, leave = True)
  mnist_correct = 0
  sum_correct = 0
  processed = 0
  for batch_idx, (data, random_digits, target, sum_of_numbers) in enumerate(pbar):
    # get samples
    data, random_digits, target, sum_of_numbers = data.to(device), random_digits.to(device), target.to(device), sum_of_numbers.to(device)

    # Init
    optimizer.zero_grad()
    # In PyTorch, we need to set the gradients to zero before starting to do backpropragation because PyTorch accumulates the gradients on subsequent backward passes. 
    # Because of this, when you start your training loop, ideally you should zero out the gradients so that you do the parameter update correctly.

    # Predict
    y_pred, sum_pred = model(data, random_digits)

    # Calculate loss
    loss_digit_detection = F.nll_loss(y_pred, target)
    loss_sum_prediction = F.nll_loss(sum_pred, sum_of_numbers)
    loss = loss_digit_detection + loss_sum_prediction
    train_losses.append(loss)

    # Backpropagation
    loss.backward()
    optimizer.step()

    # Update pbar-tqdm
    
    mnist_pred = y_pred.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
    sum_pred = sum_pred.argmax(dim=1, keepdim=True)
    mnist_correct += mnist_pred.eq(target.view_as(mnist_pred)).sum().item()
    sum_correct += sum_pred.eq(sum_of_numbers.view_as(sum_pred)).sum().item()
    processed += len(data)

    pbar.set_description(desc= f'Loss={loss.item()} Batch_id={batch_idx} Mnist_Accuracy={100*mnist_correct/processed:0.2f} Sum_Accuracy = {100*sum_correct/processed:0.2f}')
    train_acc.append(100*mnist_correct+sum_correct/2*processed)

def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    mnist_correct = 0
    sum_correct = 0
    with torch.no_grad():
        for (data, random_digits, target, sum_of_numbers) in test_loader:
                data, random_digits, target, sum_of_numbers = data.to(device), random_digits.to(device), target.to(device), sum_of_numbers.to(device)
                output, sum_pred = model(data, random_digits)
                test_loss += F.nll_loss(output, target, reduction='sum').item() + F.nll_loss(sum_pred, sum_of_numbers, reduction='sum').item() # sum up batch loss
                mnist_pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
                sum_pred = sum_pred.argmax(dim=1, keepdim=True)
                mnist_correct += mnist_pred.eq(target.view_as(mnist_pred)).sum().item()
                sum_correct += sum_pred.eq(sum_of_numbers.view_as(sum_pred)).sum().item()

    test_loss /= len(test_loader.dataset)
    test_losses.append(test_loss)

    print('\nTest set: Average loss: {:.4f}, Mnist Accuracy: {}/{}, Sum Accuracy: {}/{}, Total: ({:.2f}%)\n'.format(
        test_loss, mnist_correct, len(test_loader.dataset), sum_correct, len(test_loader.dataset),
        100. * ((mnist_correct + sum_correct) / (2*len(test_loader.dataset)))
        ))
    
    test_acc.append(100. * ((mnist_correct + sum_correct) / (2*len(test_loader.dataset))))

In [5]:
model = Net().to(device)

import torch.optim as optim
from torch.optim.lr_scheduler import StepLR
optimizer = optim.SGD(model.parameters(), lr=0.02, momentum=0.9)
scheduler = StepLR(optimizer, step_size=5, gamma=0.5)
EPOCHS = 30

for epoch in range(EPOCHS):
    print('EPOCH - ', epoch)
    train(model, device, train_loader, optimizer, epoch)
    scheduler.step()
    test(model, device, test_loader)

  0%|          | 0/118 [00:00<?, ?it/s]EPOCH -  0
Loss=2.506195545196533 Batch_id=117 Mnist_Accuracy=74.59 Sum_Accuracy = 12.73: 100%|██████████| 118/118 [00:15<00:00,  7.68it/s]
  0%|          | 0/118 [00:00<?, ?it/s]
Test set: Average loss: 2.4367, Mnist Accuracy: 9587/10000, Sum Accuracy: 1827/10000, Total: (57.07%)

EPOCH -  1
Loss=1.711471676826477 Batch_id=117 Mnist_Accuracy=95.21 Sum_Accuracy = 36.92: 100%|██████████| 118/118 [00:13<00:00,  8.67it/s]
  0%|          | 0/118 [00:00<?, ?it/s]
Test set: Average loss: 1.6883, Mnist Accuracy: 9752/10000, Sum Accuracy: 5252/10000, Total: (75.02%)

EPOCH -  2
Loss=1.0825353860855103 Batch_id=117 Mnist_Accuracy=96.68 Sum_Accuracy = 61.94: 100%|██████████| 118/118 [00:13<00:00,  8.66it/s]
  0%|          | 0/118 [00:00<?, ?it/s]
Test set: Average loss: 1.0337, Mnist Accuracy: 9820/10000, Sum Accuracy: 7781/10000, Total: (88.00%)

EPOCH -  3
Loss=0.806056797504425 Batch_id=117 Mnist_Accuracy=97.07 Sum_Accuracy = 81.18: 100%|██████████| 118/